# **Scraping eBay for sold listings of Rolex "Root Beer GMT-Master II" watches**


### ***Importing necessary modules for script below***

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandasql as ps

### ***Defining function to access ebay search URL***

##### *eBay searches model number and filters by sold items, brand-new and pre-owned conditions, and "Rolex" brand*

In [2]:
model_number = '126711CHNR'

In [3]:
def scrape_data(model_number):
    url = f"https://www.ebay.com/sch/i.html?_dcat=31387&_fsrp=1&rt=nc&_from=R40&LH_AV=1&LH_Complete=1&_ipg=240&LH_ItemCondition=3000%7C1000&LH_Sold=1&_nkw={model_number}&_sacat=0&Brand=Rolex"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

### ***Defining function to parse data from HTML of ebay search URL, arranged into preferred dictionary format***

In [4]:
def parse_data(soup):
    products_list = []
    results = soup.find_all('div', attrs={'class': 's-item__wrapper clearfix'})
    for item in results[2:243]:
        sold_listings = {
            'title': item.find('span', {'role': 'heading'}).text,
            'condition': item.find('span', {'class': 'SECONDARY_INFO'}).text,
            'sold_price_USD': item.find('span', {'class': 's-item__price'}).text,
            'sold_date': item.find('span', {'class': 's-item__caption--signal POSITIVE'}).text,
            'listing_url': item.find('a', {'class': 's-item__link'})['href']
        }
        products_list.append(sold_listings)
    return products_list

### ***Defining function to save parsed data into CSV file in library***

In [5]:
def save_csv(products_list, model_number):
    products_df = pd.DataFrame(products_list)
    products_df.to_csv('Rolex_' + model_number + '_sold_listings.csv', index=False)
    print('Rolex_' + model_number + '_sold_listings saved to CSV file')
    return

### ***Plugging variables into defined functions to create our customized CSV file of scraped eBay data***

In [6]:
soup = scrape_data(model_number)
products_list = parse_data(soup)
save_csv(products_list, model_number)

Rolex_126711CHNR_sold_listings saved to CSV file


# *Cleaning and formatting our CSV data*

### ***Reviewing our main dataframe via custom CSV file***

##### *Checking for null values and improper field data types*

In [7]:
df = pd.read_csv(f'Rolex_{model_number}_sold_listings.csv')

In [8]:
df.head()

,title,condition,sold_price_USD,sold_date,listing_url
0,Rolex GMT-Master II 126711CHNR Root Beer Rose ...,Brand New,"$20,387.00","Sold Jun 3, 2024",https://www.ebay.com/itm/126486174406?hash=ite...
1,Rolex GMT-Master II 126711CHNR Root Beer Two-T...,Pre-Owned,"$18,393.81","Sold May 31, 2024",https://www.ebay.com/itm/186449439850?hash=ite...
2,"Rolex GMT-Master II ""Root Beer"" Automatic Men'...",Brand New,"$21,525.00","Sold Jan 6, 2023",https://www.ebay.com/itm/155065004433?epid=140...
3,MINT PAPERS Rolex GMT-Master II Two-Tone Root ...,Pre-Owned,"$18,493.81","Sold May 14, 2024",https://www.ebay.com/itm/156190246811?hash=ite...
4,Rolex GMT-Master II Rootbeer Full Set Watch 12...,Pre-Owned,"$17,750.00","Sold May 13, 2024",https://www.ebay.com/itm/285852231447?hash=ite...


In [9]:
df.isnull().any()

title             False
condition         False
sold_price_USD    False
sold_date         False
listing_url       False
dtype: bool

In [10]:
df.dtypes

title             object
condition         object
sold_price_USD    object
sold_date         object
listing_url       object
dtype: object

##### *Correcting field data types and formats, adding necessary fields to the dataframe*

In [11]:
df['sold_date'] = df['sold_date'].str.strip('Sold ')
df['sold_date'] = pd.to_datetime(df['sold_date'], format='%b %d, %Y')

In [12]:
df['sold_price_USD'] = df['sold_price_USD'].str.replace('$','').str.replace(',','').astype(float)

In [13]:
df.insert(1, 'model_number', model_number)

### ***Reviewing CSV file of additional Rolex data(sourced from Kaggle)***

##### *Checking for null values and improper field data types*

In [14]:
rolex_retail = pd.read_csv('Prezzi_Originali_puliti.csv', sep=';')

In [15]:
rolex_retail.head()

,Size,Reference,Collection,Description,RRP,Complication
0,40,116900,Air King,Standard Dial,6180.0,NaN
1,40,116500LN,Cosmograph Daytona,Standard Dial,12600.0,"Chronograph, Small Seconds, Stop Seconds"
2,40,116503,Cosmograph Daytona,Standard Dial,16800.0,"Chronograph, Small Seconds"
3,40,116503,Cosmograph Daytona,Diamond Dial,18720.0,"Chronograph, Small Seconds"
4,40,116503,Cosmograph Daytona,Mother of Pearl Diamond Dial,21360.0,"Chronograph, Small Seconds"


In [16]:
rolex_retail.dtypes

Size             int64
Reference       object
Collection      object
Description     object
RRP             object
Complication    object
dtype: object

##### *Correcting field data types and formats, adding necessary fields to this dataframe*

In [17]:
rolex_retail[rolex_retail['Reference'] == model_number]

,Size,Reference,Collection,Description,RRP,Complication
152,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT"


In [18]:
rolex_retail['RRP'] = pd.to_numeric(rolex_retail['RRP'], errors='coerce').astype(float).round(2)
rolex_retail = rolex_retail.rename(columns={'RRP':'retail_GBP'})

In [19]:
GBP_USD_exch_rate = requests.get("https://api.frankfurter.app/latest?amount=1&from=gbp&to=usd").json()['rates']['USD']
rolex_retail['retail_USD'] = rolex_retail['retail_GBP'].multiply(GBP_USD_exch_rate).round(2)

# **Using SQL for additional formatting and data exploration**

### ***Joining the two tables into our main dataframe***

In [20]:
join_query = '''
    SELECT * 
    FROM df
    LEFT JOIN rolex_retail ON df.model_number = rolex_retail.Reference
    '''

df = ps.sqldf(join_query, locals())

### ***Reviewing updated dataframe***

In [21]:
df.head()

,title,model_number,condition,sold_price_USD,sold_date,listing_url,Size,Reference,Collection,Description,retail_GBP,Complication,retail_USD
0,Rolex GMT-Master II 126711CHNR Root Beer Rose ...,126711CHNR,Brand New,20387.00,2024-06-03 00:00:00.000000,https://www.ebay.com/itm/126486174406?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14
1,Rolex GMT-Master II 126711CHNR Root Beer Two-T...,126711CHNR,Pre-Owned,18393.81,2024-05-31 00:00:00.000000,https://www.ebay.com/itm/186449439850?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14
2,"Rolex GMT-Master II ""Root Beer"" Automatic Men'...",126711CHNR,Brand New,21525.00,2023-01-06 00:00:00.000000,https://www.ebay.com/itm/155065004433?epid=140...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14
3,MINT PAPERS Rolex GMT-Master II Two-Tone Root ...,126711CHNR,Pre-Owned,18493.81,2024-05-14 00:00:00.000000,https://www.ebay.com/itm/156190246811?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14
4,Rolex GMT-Master II Rootbeer Full Set Watch 12...,126711CHNR,Pre-Owned,17750.00,2024-05-13 00:00:00.000000,https://www.ebay.com/itm/285852231447?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14


##### *Adding necessary fields to the dataframe*

In [22]:
profit_query_update_df = '''
    SELECT *, sold_price_USD - retail_USD profit_USD, ((sold_price_USD / retail_USD)-1) * 100 growth_pct
    FROM df
    '''

df = ps.sqldf(profit_query_update_df, locals())
df

,title,model_number,condition,sold_price_USD,sold_date,listing_url,Size,Reference,Collection,Description,retail_GBP,Complication,retail_USD,profit_USD,growth_pct
0,Rolex GMT-Master II 126711CHNR Root Beer Rose ...,126711CHNR,Brand New,20387.00,2024-06-03 00:00:00.000000,https://www.ebay.com/itm/126486174406?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,2240.86,12.348962
1,Rolex GMT-Master II 126711CHNR Root Beer Two-T...,126711CHNR,Pre-Owned,18393.81,2024-05-31 00:00:00.000000,https://www.ebay.com/itm/186449439850?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,247.67,1.364863
2,"Rolex GMT-Master II ""Root Beer"" Automatic Men'...",126711CHNR,Brand New,21525.00,2023-01-06 00:00:00.000000,https://www.ebay.com/itm/155065004433?epid=140...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,3378.86,18.620269
3,MINT PAPERS Rolex GMT-Master II Two-Tone Root ...,126711CHNR,Pre-Owned,18493.81,2024-05-14 00:00:00.000000,https://www.ebay.com/itm/156190246811?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,347.67,1.915945
4,Rolex GMT-Master II Rootbeer Full Set Watch 12...,126711CHNR,Pre-Owned,17750.00,2024-05-13 00:00:00.000000,https://www.ebay.com/itm/285852231447?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,-396.14,-2.183054
5,MINT 2021 PAPERS Rolex GMT-Master II 18K Gold ...,126711CHNR,Pre-Owned,18393.81,2024-05-07 00:00:00.000000,https://www.ebay.com/itm/276423187207?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,247.67,1.364863
6,MINT 2021 PAPERS Rolex GMT-Master II 18K Gold ...,126711CHNR,Pre-Owned,18393.81,2024-04-30 00:00:00.000000,https://www.ebay.com/itm/305508348292?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,247.67,1.364863
7,2022 PAPERS Rolex GMT-Master II Two-Tone Root ...,126711CHNR,Pre-Owned,18993.81,2024-04-28 00:00:00.000000,https://www.ebay.com/itm/266766464267?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,847.67,4.671352
8,NEW APRIL 2024 Rolex GMT-Master II Two-Tone Ro...,126711CHNR,Brand New,19993.91,2024-04-28 00:00:00.000000,https://www.ebay.com/itm/276424263871?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,1847.77,10.182717
9,Rolex GMT-Master II 126711CHNR Root Beer Unwor...,126711CHNR,Brand New,19150.00,2024-04-20 00:00:00.000000,https://www.ebay.com/itm/135025005846?hash=ite...,40,126711CHNR,GMT-Master II,Standard Dial,14220.0,"Date, GMT",18146.14,1003.86,5.532086


### ***Data exploration***

##### *Q1: How much does a brand-new vs. pre-owned Root Beer GMT-Master II go for? How much profit is made?*

In [23]:
profit_by_cond_query = '''
    SELECT model_number, condition, ROUND(AVG(sold_price_USD), 2) avg_sold_price_USD, retail_USD, ROUND(AVG(profit_USD), 2) avg_profit_USD,
        ROUND(AVG(growth_pct), 2) avg_growth_pct
    FROM df
    GROUP BY condition
    '''

ps.sqldf(profit_by_cond_query, locals())

,model_number,condition,avg_sold_price_USD,retail_USD,avg_profit_USD,avg_growth_pct
0,126711CHNR,Brand New,20356.85,18146.14,2210.71,12.18
1,126711CHNR,Pre-Owned,18425.77,18146.14,279.63,1.54


##### *Q2: What is the distribution between brand-new vs. pre-owned listings?*

In [24]:
count_condition_query = '''
    SELECT model_number, condition, COUNT(title) count
    FROM df
    GROUP BY condition
    '''

ps.sqldf(count_condition_query, locals())

,model_number,condition,count
0,126711CHNR,Brand New,7
1,126711CHNR,Pre-Owned,13


##### *Q3: Which listings were most profitable (separated by condition)?*

In [25]:
listings_ranked_by_profit_per_cond_query = '''
    SELECT title, model_number, condition, listing_URL, sold_price_USD, retail_USD, profit_USD,
        DENSE_RANK() OVER (PARTITION BY condition ORDER BY profit_USD DESC) ranked_profit_by_cond
    FROM df
    '''

ps.sqldf(listings_ranked_by_profit_per_cond_query, locals())

,title,model_number,condition,listing_url,sold_price_USD,retail_USD,profit_USD,ranked_profit_by_cond
0,"Rolex GMT-Master II ""Root Beer"" Automatic Men'...",126711CHNR,Brand New,https://www.ebay.com/itm/155065004433?epid=140...,21525.00,18146.14,3378.86,1
1,NEW FEB 2024 PAPERS Rolex GMT-Master II 18K Go...,126711CHNR,Brand New,https://www.ebay.com/itm/276346726913?hash=ite...,20993.02,18146.14,2846.88,2
2,Rolex GMT-Master II 126711CHNR Root Beer UNWOR...,126711CHNR,Brand New,https://www.ebay.com/itm/186290192932?hash=ite...,20950.00,18146.14,2803.86,3
3,Rolex GMT-Master II 126711CHNR Root Beer Rose ...,126711CHNR,Brand New,https://www.ebay.com/itm/126486174406?hash=ite...,20387.00,18146.14,2240.86,4
4,NEW APRIL 2024 Rolex GMT-Master II Two-Tone Ro...,126711CHNR,Brand New,https://www.ebay.com/itm/276424263871?hash=ite...,19993.91,18146.14,1847.77,5
5,2022 Unworn Unsized Rolex GMT Master II Two To...,126711CHNR,Brand New,https://www.ebay.com/itm/145338143533?hash=ite...,19499.00,18146.14,1352.86,6
6,Rolex GMT-Master II 126711CHNR Root Beer Unwor...,126711CHNR,Brand New,https://www.ebay.com/itm/135025005846?hash=ite...,19150.00,18146.14,1003.86,7
7,2022 PAPERS Rolex GMT-Master II Two-Tone Root ...,126711CHNR,Pre-Owned,https://www.ebay.com/itm/266766464267?hash=ite...,18993.81,18146.14,847.67,1
8,2022 Rolex GMT-Master II Two-Tone ROOT BEER Ro...,126711CHNR,Pre-Owned,https://www.ebay.com/itm/266729960881?hash=ite...,18993.81,18146.14,847.67,1
9,Rolex GMT-Master II Root Beer Wristwatch 12671...,126711CHNR,Pre-Owned,https://www.ebay.com/itm/186342568300?hash=ite...,18905.00,18146.14,758.86,2


##### *Q4: What are the monthly averages for sales per condition?*

In [26]:
monthly_avgs_by_condition_query = '''
    SELECT model_number, condition, strftime('%Y-%m', sold_date) month, ROUND(AVG(sold_price_USD), 2) avg_sold_price_USD, retail_USD, 
        ROUND(AVG(profit_USD), 2) avg_profit_USD
    FROM df
    GROUP BY condition, month
    ORDER BY condition, month DESC
    '''

ps.sqldf(monthly_avgs_by_condition_query, locals())

,model_number,condition,month,avg_sold_price_USD,retail_USD,avg_profit_USD
0,126711CHNR,Brand New,2024-06,20387.00,18146.14,2240.86
1,126711CHNR,Brand New,2024-04,20031.30,18146.14,1885.16
2,126711CHNR,Brand New,2024-03,20246.01,18146.14,2099.87
3,126711CHNR,Brand New,2023-01,21525.00,18146.14,3378.86
4,126711CHNR,Pre-Owned,2024-05,18257.86,18146.14,111.72
5,126711CHNR,Pre-Owned,2024-04,18693.81,18146.14,547.67
6,126711CHNR,Pre-Owned,2024-03,18445.14,18146.14,299.00


### ***Saving finalized dataframe as CSV file to library***

In [27]:
def save_final_csv(final_df,model_number):
    final_df.to_csv('Rolex_' + model_number + '_sold_listings_final.csv', index=False)
    print('Rolex_' + model_number + '_sold_listings_final saved to CSV file')
    return

save_final_csv(df,model_number)

Rolex_126711CHNR_sold_listings_final saved to CSV file
